In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%pip install -q decorator==5.0.9
!pip install -q ipywidgets

import csv
import math
import re
import imp
import json
import numpy as np
import pandas as pd
import difflib as dl
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from collections import Counter

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q cltk==1.0.22
%pip install -q strsim
%pip install -q leven

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# importing local modules

import sys
sys.path.append('../src/')

import text_cleanup.text_cleanup as thesisCleanUp
import preprocessing.text_preprocessing as thesisTextPreprocessing
import data.reader as thesisDataReader
import utils.utils as thesisUtils
import features.tf_idf.n_gram as thesisTfIdfNgramFeatures
import features.count_vectorizer.n_gram as thesisCountVectorizerNgramFeatures
import similarities.cosine as thesisCosineSimilarities
import similarities.levenshtein as thesisLevenshteinSimilarities
import vocabulary.vocabulary as thesisVocabulary
import features.lexical as thesisLexicalFeatures
import similarities.cosine as thesisCosineSimilarity
import text_cleanup.text_cleanup as thesisTextCleanUp
import p_aligment.p_aligment as thesisPAligment
import features.model_features as thesisModelFeatures

imp.reload(thesisTfIdfNgramFeatures)
imp.reload(thesisLexicalFeatures)
imp.reload(thesisCosineSimilarity)
imp.reload(thesisCleanUp)
imp.reload(thesisTextPreprocessing)
imp.reload(thesisDataReader)
imp.reload(thesisUtils)

imp.reload(thesisVocabulary)
imp.reload(thesisCosineSimilarities)
imp.reload(thesisTextCleanUp)
imp.reload(thesisCountVectorizerNgramFeatures)
imp.reload(thesisPAligment)
imp.reload(thesisLevenshteinSimilarities)
imp.reload(thesisModelFeatures)

<module 'features.model_features' from '../src/features/model_features.py'>

# Create burchard version in original london format

1. create london/zwickau versions without words/letters replacement - replace only numbers, punctuation, etc...
2. create word mapping of a new created versions in 1 to regular processed versions
3. change words in burchard candidate according to mapping in 2 - it should return us burchard in original format

In [5]:
london_corpus_by_new_line_without_words_processing = thesisDataReader.get_london_by_new_line_without_words_processing()
zwickau_corpus_by_new_line_without_words_processing = thesisDataReader.get_zwickau_by_new_line_without_words_processing()

In [6]:
london_original_to_processed_mapping = thesisVocabulary.create_pre_post_processing_map(
    ' \n'.join(london_corpus_by_new_line_without_words_processing)
)

In [7]:
burchard_version_with_original_london_text = []

for p in thesisDataReader.get_burchard_candidate_version_based_on_strongly_similar_london_base():
    new_p = ''
    
    for word in p.split():
        original_word = london_original_to_processed_mapping[london_original_to_processed_mapping['after'] == word]['before'].values[0]
        new_p = f'{new_p} {original_word}' if len(new_p) > 0 else original_word
    
    burchard_version_with_original_london_text.append(new_p)

# td_idf_3_gram + cosine_similarity

In [8]:
features_tfidf_3_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 3,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

In [9]:
features_tfidf_3_gram_cosine_similarity_df

,index,version,a,aa,ab,ac,ad,ae,af,ag,...,zoc,zoe,zor,zot,zra,zuc,zur,zyd,zyp,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065368
1,1.0,0.0,0.0,0.05966,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.268126
2,2.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.209745
3,3.0,0.0,0.0,0.00000,0.000000,0.000000,0.028476,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.243578
4,4.0,0.0,0.0,0.00000,0.021612,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.230167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.00000,0.000000,0.032224,0.033061,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.198400
634,317.0,1.0,0.0,0.00000,0.013060,0.017769,0.036460,0.033796,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.311705
635,318.0,1.0,0.0,0.00000,0.000000,0.000000,0.023401,0.000000,0.0,0.021638,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.241369
636,319.0,1.0,0.0,0.00000,0.027644,0.000000,0.057881,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.210448


In [10]:
tf_idf_3_cosine_results = thesisModelFeatures.run_models(features_tfidf_3_gram_cosine_similarity_df)

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [52]:
tf_idf_3_cosine_results.sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
AdaBoostClassifier,0.531445,0.531341,0.528163,0.531161,0.528095,0.531161
DecisionTreeClassifier,0.527983,0.525453,0.520624,0.525222,0.520589,0.525222
SVM_linear,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
GaussianProcessClassifier,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
MLPClassifier,0.457567,0.466123,0.446537,0.466622,0.446824,0.466622
QuadraticDiscriminantAnalysis,0.447927,0.449185,0.444677,0.449379,0.444817,0.449379
GaussianNB,0.383837,0.387319,0.382238,0.387145,0.382156,0.387145
KNeighborsClassifier,0.377841,0.382518,0.376983,0.382668,0.377117,0.382668
RandomForestClassifier,0.348761,0.349457,0.348401,0.349335,0.348345,0.349335
SVM_RBF,0.285457,0.288587,0.286011,0.288608,0.286048,0.288608


# td_idf_4_gram + cosine_similarity


In [12]:
features_tfidf_4_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 4,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

In [13]:
features_tfidf_4_gram_cosine_similarity_df

,index,version,a a,a b,a c,a d,a f,a g,a i,a l,...,zra,zrae,zrah,zuca,zucc,zucu,zuri,zyda,zyph,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021962
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.086926
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066421
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083912
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.072329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062225
634,317.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.118377
635,318.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087864
636,319.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062714


In [14]:
tf_idf_4_cosine_results = thesisModelFeatures.run_models(features_tfidf_4_gram_cosine_similarity_df)

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [53]:
tf_idf_4_cosine_results.sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
AdaBoostClassifier,0.575515,0.573460,0.571214,0.573404,0.571214,0.573404
DecisionTreeClassifier,0.541333,0.540036,0.537011,0.539672,0.536853,0.539672
SVM_linear,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
GaussianProcessClassifier,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
MLPClassifier,0.413188,0.423007,0.407690,0.422606,0.407452,0.422606
KNeighborsClassifier,0.392221,0.393297,0.390787,0.393174,0.390735,0.393174
QuadraticDiscriminantAnalysis,0.390923,0.393116,0.390093,0.393129,0.390118,0.393129
RandomForestClassifier,0.368355,0.370380,0.368460,0.370124,0.368297,0.370124
GaussianNB,0.345291,0.347645,0.344855,0.347429,0.344743,0.347429
SVM_RBF,0.244225,0.254982,0.247508,0.255142,0.247649,0.255142


# td_idf_5_gram + cosine_similarity

In [8]:
features_tfidf_5_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 5,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

In [9]:
features_tfidf_5_gram_cosine_similarity_df

,index,corpus_version_label,a aq,a ba,a be,a ca,a ce,a ch,a ci,a cr,...,zra d,zrael,zrahe,zucar,zucca,zucur,zurio,zyda,zyph,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009855
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.032757
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025986
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033445
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024990
634,317.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050199
635,318.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034937
636,319.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023450


In [18]:
tf_idf_5_cosine_results = thesisModelFeatures.run_models(features_tfidf_5_gram_cosine_similarity_df)

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [54]:
tf_idf_5_cosine_results.sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
AdaBoostClassifier,0.580237,0.579438,0.577976,0.579477,0.578012,0.579477
DecisionTreeClassifier,0.561815,0.561051,0.559533,0.560771,0.559439,0.560771
SVM_linear,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
GaussianProcessClassifier,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
QuadraticDiscriminantAnalysis,0.425054,0.428261,0.423320,0.428635,0.423572,0.428635
KNeighborsClassifier,0.402640,0.405797,0.401431,0.405718,0.401407,0.405718
RandomForestClassifier,0.368142,0.372101,0.366737,0.372163,0.366794,0.372163
MLPClassifier,0.358734,0.370109,0.353283,0.369902,0.353151,0.369902
GaussianNB,0.275304,0.280888,0.275933,0.280496,0.275684,0.280496
SVM_RBF,0.236190,0.259058,0.242737,0.259309,0.242947,0.259309


# td_idf_6_gram + cosine_similarity

In [23]:
features_tfidf_6_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 6,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

In [35]:
features_tfidf_6_gram_cosine_similarity_df

,index,version,a aqu,a bab,a bel,a ber,a bet,a car,a cas,a ced,...,zra di,zrael,zrahel,zucara,zuccar,zucura,zurio,zyda o,zyph d,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005269
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014145
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011290
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015839
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011393
634,317.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023521
635,318.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015032
636,319.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010123


In [25]:
tf_idf_6_cosine_results = thesisModelFeatures.run_models(features_tfidf_6_gram_cosine_similarity_df)

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [55]:
tf_idf_6_cosine_results.sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
AdaBoostClassifier,0.570020,0.567029,0.563705,0.566755,0.563599,0.566755
DecisionTreeClassifier,0.523604,0.523098,0.519355,0.522872,0.519250,0.522872
SVM_linear,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
GaussianProcessClassifier,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
KNeighborsClassifier,0.426798,0.428804,0.425640,0.428679,0.425593,0.428679
RandomForestClassifier,0.412124,0.413949,0.411365,0.414229,0.411555,0.414229
QuadraticDiscriminantAnalysis,0.398539,0.403804,0.397240,0.403768,0.397240,0.403768
MLPClassifier,0.334309,0.351087,0.334793,0.351596,0.335132,0.351596
SVM_RBF,0.216368,0.258877,0.230008,0.259397,0.230419,0.259397
GaussianNB,0.245513,0.251268,0.246909,0.251064,0.246772,0.251064


# td_idf_7_gram + cosine_similarity

In [43]:
features_tfidf_7_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 7,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

In [56]:
features_tfidf_7_gram_cosine_similarity_df

,index,corpus_version_label,a aque,a baby,a belu,a bers,a beth,a carm,a cast,a casu,...,zrahel,zraheli,zucara,zuccara,zucura,zurio e,zurio i,zyda op,zyph di,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002966
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006549
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005263
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008438
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005794
634,317.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011862
635,318.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006710
636,319.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004590


In [45]:
tf_idf_7_cosine_results = thesisModelFeatures.run_models(features_tfidf_7_gram_cosine_similarity_df)

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [57]:
tf_idf_7_cosine_results.sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
AdaBoostClassifier,0.566896,0.564764,0.562523,0.564761,0.562554,0.564761
SVM_linear,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
GaussianProcessClassifier,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
DecisionTreeClassifier,0.495771,0.496014,0.492093,0.495922,0.492075,0.495922
QuadraticDiscriminantAnalysis,0.454923,0.455888,0.451864,0.456206,0.452017,0.456206
KNeighborsClassifier,0.450430,0.451630,0.449470,0.451729,0.449582,0.451729
RandomForestClassifier,0.371483,0.382246,0.370746,0.382713,0.371090,0.382713
MLPClassifier,0.324309,0.334692,0.319865,0.334353,0.319625,0.334353
SVM_RBF,0.190340,0.248188,0.211145,0.248892,0.211717,0.248892
GaussianNB,0.234985,0.240580,0.236375,0.240514,0.236342,0.240514


# td_idf_8_gram + cosine_similarity

In [47]:
features_tfidf_8_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 8,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

In [48]:
features_tfidf_8_gram_cosine_similarity_df

,index,corpus_version_label,a aque,a babyl,a belua,a bersa,a betha,a bethe,a beths,a bethu,...,zraheli,zrahelit,zucara m,zuccara,zucura e,zurio et,zurio it,zyda opi,zyph dic,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001788
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003370
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002508
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004578
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003071
634,317.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006157
635,318.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003205
636,319.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002167


In [49]:
tf_idf_8_cosine_results = thesisModelFeatures.run_models(features_tfidf_8_gram_cosine_similarity_df)

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [58]:
tf_idf_8_cosine_results.sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
DecisionTreeClassifier,0.531522,0.527717,0.524416,0.527615,0.524392,0.527615
AdaBoostClassifier,0.510461,0.510507,0.506681,0.510151,0.506502,0.510151
SVM_linear,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
GaussianProcessClassifier,0.251064,0.500000,0.334272,0.502128,0.335711,0.502128
QuadraticDiscriminantAnalysis,0.495944,0.496014,0.489859,0.495833,0.489801,0.495833
KNeighborsClassifier,0.440074,0.440851,0.439468,0.441046,0.439623,0.441046
RandomForestClassifier,0.362213,0.384239,0.359202,0.384929,0.359662,0.384929
MLPClassifier,0.309367,0.339130,0.318179,0.338741,0.317917,0.338741
SVM_RBF,0.177919,0.243931,0.203309,0.244681,0.203931,0.244681
GaussianNB,0.219864,0.225815,0.221614,0.225709,0.221535,0.225709


# tf_idf_5_gram + cosine similarity GridSearchCV

In [141]:
imp.reload(thesisModelFeatures)

<module 'features.model_features' from '../src/features/model_features.py'>

In [96]:
resp = thesisModelFeatures.run_grid_search_cv(features_tfidf_5_gram_cosine_similarity_df)

In [97]:
resp

GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'gamma': array([2.09205021e-06, 2.09205021e-05, 2.09205021e-04, 2.09205021e-03,
       2.09205021e-02, 2.09205021e-01]),
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             return_train_score=True)

In [111]:
scores_df = pd.DataFrame(dtype=float)
classifier_name = 'SVC'

In [112]:
scores_df.loc[classifier_name, 'best_score_'] = resp.best_score_
scores_df.loc[classifier_name, 'best_estimator_'] = resp.best_estimator_

In [108]:
scores_df

,best_score_,best_estimator_
SVC,0.502128,"SVC(C=0.001, gamma=2.092050209205021e-06, kern..."


In [151]:
imp.reload(thesisModelFeatures)

<module 'features.model_features' from '../src/features/model_features.py'>

In [118]:
respDecisionTreeClassifier, gridsDecisionTreeClassifier = thesisModelFeatures.run_grid_search_cv(features_tfidf_5_gram_cosine_similarity_df)

running: DecisionTreeClassifier


In [119]:
respDecisionTreeClassifier

,best_score_,best_estimator_
DecisionTreeClassifier,0.658865,"DecisionTreeClassifier(criterion='entropy', ma..."


In [123]:
respGaussianProcessClassifier, gridsGaussianProcessClassifier = thesisModelFeatures.run_grid_search_cv(features_tfidf_5_gram_cosine_similarity_df)

running: GaussianProcessClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


In [124]:
respGaussianProcessClassifier

,best_score_,best_estimator_
GaussianProcessClassifier,0.502128,GaussianProcessClassifier(kernel=1**2 * RBF(le...


In [128]:
respRandomForestClassifier, gridsRandomForestClassifier = thesisModelFeatures.run_grid_search_cv(features_tfidf_5_gram_cosine_similarity_df)

running: RandomForestClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.49986702 0.49366135 0.48324468 0.5        0.48727837 0.49148936
 0.48945035 0.49782801 0.49986702 0.49366135 0.48324468 0.5
 0.48727837 0.49148936 0.48945035 0.49782801 0.44343972 0.40155142
 0.4016844  0.39968972 0.39117908 0.37238475 0.36387411 0.36409574
 0.48098404 0.49778369 0.48541667 0.4748227  0.48315603 0.47273936
 0.4748227  0.49370567 0.48098404 0.49778369 0.48541667 0.4748227
 0.48315603 0.47273936 0.4748227  0.49370567 0.38067376 0.38266844
 0.37659574 0.37655142 0.39530142 0.35766844 0.35970745 0.35345745
 0.46852837 0.47681738 0.46001773 0.46218972 0.48931738 0.49782801
 0.48736702 0.48532801 0.46852837 0.47681738 0.46001773 0.46218972
 0.48931738 0.49782801 0.48736702 0.48532801 0.40376773 0.37637411
 0.37437943 0.36609043 0.37234043 0.35545213 0.34920213 0.34299645
 0.45159574 0.47047872 0.43519504 0.4538

running: KNeighborsClassifier


In [129]:
respRandomForestClassifier

""


In [132]:
gridsRandomForestClassifier[0].best_score_

0.5

In [133]:
gridsRandomForestClassifier[1].best_score_

0.40571808510638296

In [135]:
respMLPClassifier, gridsMLPClassifier = thesisModelFeatures.run_grid_search_cv(features_tfidf_5_gram_cosine_similarity_df)

running: MLPClassifier


KeyboardInterrupt: 

In [137]:
respGaussianNB, gridsGaussianNB = thesisModelFeatures.run_grid_search_cv(features_tfidf_5_gram_cosine_similarity_df)

running: GaussianNB


In [140]:
gridsGaussianNB[0].best_score_

0.28891843971631204

In [142]:
respKNeighborsClassifier, gridsKNeighborsClassifier = thesisModelFeatures.run_grid_search_cv(features_tfidf_5_gram_cosine_similarity_df)

running: KNeighborsClassifier


In [143]:
gridsKNeighborsClassifier[0].best_score_

0.5105939716312058

In [146]:
respAdaBoostClassifier, gridsAdaBoostClassifier = thesisModelFeatures.run_grid_search_cv(features_tfidf_5_gram_cosine_similarity_df)

running: AdaBoostClassifier


In [147]:
gridsAdaBoostClassifier[0].best_score_

0.7152482269503546

In [ ]:
respQuadraticDiscriminantAnalysis, gridsQuadraticDiscriminantAnalysis = thesisModelFeatures.run_grid_search_cv(features_tfidf_5_gram_cosine_similarity_df)

running: QuadraticDiscriminantAnalysis


In [ ]:
gridsQuadraticDiscriminantAnalysis[0].best_score_